<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/File_Share_6_Fetching_the_time_table_of_a_given_stop_point_of_given_lineID%2C_direction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fetching Time table and dividing to slots- (changing 24 to 00) and returning the slot corresponding to present hour

In [1]:
import requests
import pandas as pd
from datetime import datetime
import pytz

# Function to fetch and display timetable
def fetch_and_display_timetable(line_id, stop_point_id, direction):
    try:
        url = f"https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the timetable
        schedules = data.get('timetable', {}).get('routes', [{}])[0].get('schedules', [])
        if not schedules:
            print("No timetable available.")
            return

        # Create a list of dictionaries to store timetable entries
        timetable_entries = []
        for schedule in schedules:
            for entry in schedule.get('knownJourneys', []):
                timetable_entries.append({
                    'Hour': int(entry['hour']),
                    'Minute': int(entry['minute']),
                    'Line ID': line_id,
                    'Stop Point ID': stop_point_id,
                    'Direction': direction.capitalize()
                })

        # Create a DataFrame and display it in a readable format
        df = pd.DataFrame(timetable_entries)
        df = df.sort_values(by=['Hour', 'Minute'])
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Slot assignment
        df['Slot'] = df['Hour'] + 1
        df['Slot'] = df['Slot'].apply(lambda x: x if x <= 24 else x - 24)

        # Adjust hours to range 00-23
        df['Hour'] = df['Hour'].apply(lambda x: x if x < 24 else 0)
        df['Time'] = df.apply(lambda row: f"{row['Hour']:02d}:{row['Minute']:02d}", axis=1)

        # Get current hour in BST
        bst_tz = pytz.timezone('Europe/London')
        current_time = datetime.now(bst_tz)
        current_hour = current_time.hour

        # Determine current slot
        current_slot = current_hour + 1
        if current_slot > 24:
            current_slot = 1

        # Display timetable for the current slot
        slot_start = (current_slot - 1) % 24
        slot_end = current_slot % 24
        print(f"\nCurrent Slot {current_slot:02d} (Hour {slot_start:02d}:00 to Hour {slot_end:02d}:00)")
        slot_df = df[df['Slot'] == current_slot]
        if not slot_df.empty:
            print(slot_df[['Time', 'Line ID', 'Stop Point ID', 'Direction']].to_string(index=False))
        else:
            print("No entries")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    stop_point_id = input("Enter stop point ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_and_display_timetable(line_id, stop_point_id, direction)


Enter line ID: 179
Enter stop point ID: 490005183E
Enter direction (e.g., inbound or outbound): outbound

Current Slot 16 (Hour 15:00 to Hour 16:00)
 Time Line ID Stop Point ID Direction
15:01     179    490005183E  Outbound
15:01     179    490005183E  Outbound
15:02     179    490005183E  Outbound
15:12     179    490005183E  Outbound
15:13     179    490005183E  Outbound
15:13     179    490005183E  Outbound
15:13     179    490005183E  Outbound
15:24     179    490005183E  Outbound
15:24     179    490005183E  Outbound
15:25     179    490005183E  Outbound
15:32     179    490005183E  Outbound
15:35     179    490005183E  Outbound
15:35     179    490005183E  Outbound
15:37     179    490005183E  Outbound
15:47     179    490005183E  Outbound
15:47     179    490005183E  Outbound
15:49     179    490005183E  Outbound
15:51     179    490005183E  Outbound
15:57     179    490005183E  Outbound
15:57     179    490005183E  Outbound
